In [17]:
import pandas as pd
from openpyxl import load_workbook

In [18]:
# Function to import the data from the Excel file
def load_datasets(benchmark_file='Benchmark.csv', target_file='Target.csv'):
    # Read CSV, using comma as the decimal separator
    df_benchmark = pd.read_csv(benchmark_file, delimiter=';', decimal=',')
    df_target = pd.read_csv(target_file, delimiter=';', decimal=',')

    # Convert all columns to numeric, coercing errors to NaN
    df_benchmark = df_benchmark.apply(pd.to_numeric, errors='coerce')
    df_target = df_target.apply(pd.to_numeric, errors='coerce')

    return df_benchmark, df_target

In [19]:
# Function to calculate the necessary metrics for both datasets
def calculate_metrics(df, target_size, weights):
    metrics = {}

    # Handle 'collection' and 'payments' separately
    if 'Payments' in df.columns:
        total_paid = df['Payments'].sum()  # Total paid
        total_debt = df['Debt'].sum()  # Total debt
        metrics['collection'] = total_paid / (total_debt + total_paid) if total_debt + total_paid != 0 else 0  # % of collection
        metrics['payments'] = df['Payments'].notnull().sum() / len(df)
    elif 'LP' in df.columns:
        metrics['collection'] = None
        metrics['payments'] = df['LP'].notnull().sum() / len(df)
    else:
        metrics['collection'] = None
        metrics['payments'] = None

    # Calculate mean for each metric in weights if the column exists
    for column in weights.keys():
        if column in ['collection', 'payments']:
            continue  # already handled
        metrics[column] = df[column].mean(skipna=True) if column in df.columns else None   # Calculate averages for all columns, skipping empty cells

    return metrics

In [20]:

# Objective function to minimize the difference between benchmark and target metrics
def objective_function(x, df_benchmark, df_target, target_metrics, weights):
    selected_indices = [int(i) for i in x]  # Convert row numbers to integers
    sample = df_benchmark.iloc[selected_indices]  # Select rows based on indices
    sample_metrics = calculate_metrics(sample, len(selected_indices), weights)  # Calculate metrics for sample

    # Calculate the weighted percentage difference
    percentage_diff = 0
    for key in weights.keys():
        percentage_diff += weights[key] * abs((sample_metrics[key] - target_metrics[key]) / target_metrics[key]) * 100

    return percentage_diff

In [21]:
# Greedy Sampling function to ensure uniqueness
def greedy_sampling(df_benchmark, df_target, weights, target_size):
    target_metrics = calculate_metrics(df_target, target_size, weights)
    selected_indices = set()  # Using a set to keep track of unique indices

    # Greedy approach: iteratively select the best row minimizing the difference
    while len(selected_indices) < target_size:
        best_index = None
        best_diff = float('inf')

        for i in range(len(df_benchmark)):
            if i in selected_indices:
                continue  # Skip if already selected

            temp_indices = selected_indices.union({i})
            temp_sample = df_benchmark.iloc[list(temp_indices)]
            temp_metrics = calculate_metrics(temp_sample, len(temp_indices), weights)
            temp_diff = sum(weights[key] * abs((temp_metrics[key] - target_metrics[key]) / target_metrics[key]) * 100
                        for key in weights
                        if temp_metrics.get(key) is not None and target_metrics.get(key) is not None and target_metrics.get(key) != 0)


            if temp_diff < best_diff:
                best_diff = temp_diff
                best_index = i

        if best_index is not None:
            selected_indices.add(best_index)

    return list(selected_indices)

In [22]:
# Function to write results to a new CSV
def write_results_to_csv(df_benchmark, selected_indices, benchmark_file='Benchmark.csv'):
    # Determine the new sample column name (e.g., Sample 1, Sample 2, etc.)
    sample_cols = [col for col in df_benchmark.columns if str(col).startswith("Sample")]
    new_sample_col = f'Sample {len(sample_cols) + 1}'

    # Add the new column initialized to 0
    df_benchmark[new_sample_col] = 0

    # Set the value to 1 for selected rows
    df_benchmark.loc[selected_indices, new_sample_col] = 1

    # Write updated dataframe back to CSV
    df_benchmark.to_csv(benchmark_file, sep=';', index=False)

In [23]:
# Main function to run the process
def main():
    df_benchmark, df_target = load_datasets()

    # Define the column weights
    weights = {
        'Debt': 0.20,
        'GR': 0.05,
        'LP': 0.10,        
        'DPD': 0.10,
        'collection': 0.15,
        'payments': 0.10
    }


    # Perform greedy sampling to select the best rows
    target_size = 50
    selected_indices = greedy_sampling(df_benchmark, df_target, weights, target_size)

    # Write the results back to the Excel file, adding a new sample column
    write_results_to_csv(df_benchmark, selected_indices)

In [24]:
if __name__ == "__main__": 
    main()